# Section: Encrypted Deep Learning

- Lesson: Reviewing Additive Secret Sharing
- Lesson: Encrypted Subtraction and Public/Scalar Multiplication
- Lesson: Encrypted Computation in PySyft
- Project: Build an Encrypted Database
- Lesson: Encrypted Deep Learning in PyTorch
- Lesson: Encrypted Deep Learning in Keras
- Final Project

# Lesson: Reviewing Additive Secret Sharing

_For more great information about SMPC protocols like this one, visit https://mortendahl.github.io. With permission, Morten's work directly inspired this first teaching segment._

In [1]:
import random
import numpy as np

BASE = 10

PRECISION_INTEGRAL = 8
PRECISION_FRACTIONAL = 8
Q = 293973345475167247070445277780365744413

PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL

assert(Q > BASE**PRECISION)

def encode(rational):
    upscaled = int(rational * BASE**PRECISION_FRACTIONAL)
    field_element = upscaled % Q
    return field_element

def decode(field_element):
    upscaled = field_element if field_element <= Q/2 else field_element - Q
    rational = upscaled / BASE**PRECISION_FRACTIONAL
    return rational

def encrypt(secret):
    first  = random.randrange(Q)
    second = random.randrange(Q)
    third  = (secret - first - second) % Q
    return [first, second, third]

def decrypt(sharing):
    return sum(sharing) % Q

def add(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [2]:
x = encrypt(encode(5.5))
x

[40077646724924128873381492968866835633,
 9374835291417446082744758152360296387,
 244520863458825672114319026659688612393]

In [3]:
y = encrypt(encode(2.3))
y

[117676205205812112639914884324599131269,
 158009036002557149913135604810503231754,
 18288104266797984517394788645493381389]

In [4]:
z = add(x,y)
z

(157753851930736241513296377293465966902,
 167383871293974595995880362962863528141,
 262808967725623656631713815305181993782)

In [5]:
decode(decrypt(z))

7.79999999

# Lesson: Encrypted Subtraction and Public/Scalar Multiplication

In [6]:
field = 23740629843760239486723

In [7]:
x = 5

bob_x_share = 2372385723 # random number
alices_x_share = field - bob_x_share + x

In [8]:
(bob_x_share + alices_x_share) % field

5

In [9]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [10]:
((bob_x_share + alice_x_share) - (bob_y_share + alice_y_share)) % field

4

In [11]:
((bob_x_share - bob_y_share) + (alice_x_share - alice_y_share)) % field

4

In [12]:
bob_x_share + alice_x_share + bob_y_share + alice_y_share

26

In [13]:
bob_z_share = (bob_x_share - bob_y_share)
alice_z_share = (alice_x_share - alice_y_share)

In [14]:
(bob_z_share + alice_z_share) % field

4

In [15]:
def sub(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] - b[i]) % Q)
    return tuple(c)

In [16]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [17]:
bob_x_share + alice_x_share

15

In [18]:
bob_y_share + alice_y_share

11

In [19]:
((bob_y_share * 3) + (alice_y_share * 3)) % field

3

In [20]:
def imul(a, scalar):
    
    # logic here which can multiply by a public scalar
    
    c = list()
    
    for i in range(len(a)):
        # Works without `% Q`
        c.append((a[i] * scalar) % Q)
        
    return tuple(c)

In [21]:
x = encrypt(encode(5.5))
x

[27245081263251511360499411370538911412,
 226493552975798874772170459951892264416,
 40234711236116860937775406458484568585]

In [22]:
z = imul(x, 3)
z

(81735243789754534081498234111616734236,
 91533967977062130175620824294945304422,
 120704133708350582813326219375453705755)

In [23]:
decode(decrypt(z))

16.5

# Lesson: Encrypted Computation in PySyft

In [24]:
import syft as sy
import torch
hook = sy.TorchHook(torch)
from torch import nn, optim

In [25]:
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

In [26]:
x = torch.tensor([1,2,3,4])
y = torch.tensor([2,-1,1,0])

In [27]:
x = x.share(bob, alice, crypto_provider=secure_worker)

In [28]:
y = y.share(bob, alice, crypto_provider=secure_worker)

In [29]:
z = x + y
z.get()

tensor([3, 1, 4, 4])

In [30]:
z = x - y
z.get()

tensor([-1,  3,  2,  4])

In [31]:
z = x * y
z.get()

tensor([ 2, -2,  3,  0])

In [32]:
z = x > y
z.get()

tensor([0, 1, 1, 1])

In [33]:
z = x < y
z.get()

tensor([1, 0, 0, 0])

In [34]:
z = x == y
z.get()

tensor([0, 0, 0, 0])

In [35]:
x = torch.tensor([1,2,3,4])
y = torch.tensor([2,-1,1,0])

x = x.fix_precision().share(bob, alice, crypto_provider=secure_worker)
y = y.fix_precision().share(bob, alice, crypto_provider=secure_worker)

In [36]:
z = x + y
z.get().float_precision()

tensor([3., 1., 4., 4.])

In [37]:
z = x - y
z.get().float_precision()

tensor([-1.,  3.,  2.,  4.])

In [38]:
z = x * y
z.get().float_precision()

tensor([ 2., -2.,  3.,  0.])

In [39]:
z = x > y
z.get().float_precision()

tensor([0., 1., 1., 1.])

In [40]:
z = x < y
z.get().float_precision()

tensor([1., 0., 0., 0.])

In [41]:
z = x == y
z.get().float_precision()

tensor([0., 0., 0., 0.])

# Project: Build an Encrypted Database

In [42]:
import string

In [43]:
char2int = {}
int2char = {}

for i, c in enumerate(' ' + string.ascii_lowercase + string.digits + string.punctuation):
    char2int[c] = i
    int2char[i] = c

In [44]:
def string2tensor(str_input, max_len=None, padding_char=' ', one_hot=True, dtype=torch.int64):
    assert (max_len is None) or (isinstance(max_len, int) and max_len >= 0)
    assert isinstance(padding_char, str) and (len(padding_char) == 1)

    str_input = str_input.lower()
    if max_len:
        if len(str_input) >= max_len:
            str_input = str_input[:max_len]
        else:
            str_input = str_input + padding_char * (max_len - len(str_input))
            
    ints = [char2int[c] for c in str_input]
    
    if one_hot:
        tensor = torch.zeros(len(str_input), len(char2int), dtype=dtype)
        tensor[torch.arange(len(str_input), dtype=torch.long), ints] = 1
    else:
        tensor = torch.tensor(ints, dtype=dtype)
    
    return tensor

In [45]:
class EncryptedStringDB(object):
    
    def __init__(self, *owners, max_key_len=8, max_value_len=8, padding_char=' '):
        assert len(owners) > 0

        self.owners        = owners
        self.max_key_len   = max_key_len
        self.max_value_len = max_value_len
        self.padding_char  = padding_char
        
        self.keys   = []
        self.values = []
    
    def add_entry(self, key, value):
        assert isinstance(key, str)
        assert isinstance(value, str)
        
        key_tensor   = string2tensor(key, self.max_key_len, self.padding_char, one_hot=True, dtype=torch.int64).share(*self.owners)

        if (len(self.keys) != 0) and sum([key_match for key_match in self._find_key_matches(key_tensor)]).get().item() == 1:
            raise NotImplementedError('adding an already existing key is not supported')
        
        value_tensor = string2tensor(value, self.max_value_len, self.padding_char, one_hot=False, dtype=torch.int64).share(*self.owners)
        
        self.keys.append(key_tensor)
        self.values.append(value_tensor)
    
    def query(self, query_str):
        assert isinstance(query_str, str)
        
        query_matrix = string2tensor(query_str, self.max_key_len, one_hot=True, dtype=torch.int64)
        query_matrix = query_matrix.share(*self.owners)
        
        key_matches = self._find_key_matches(query_matrix)
        
        if sum([key_match.clone() for key_match in key_matches]).get().item() == 0:
            raise KeyError('no match found for the query \'{}\''.format(query_str))
        
        result = torch.stack([key_match * value for key_match, value in zip(key_matches, self.values)]).sum(0)

        result = result.get()
        
        result_str = ''.join([int2char[int(char_int)] for char_int in result])
        
        return result_str
    
    def _find_key_matches(self, query_matrix):
        key_matches = []
        
        for key in self.keys:
            match_vec = (key * query_matrix).sum(1)
            
            x = match_vec[0]
            for idx in range(1, len(match_vec)):
                x = x * match_vec[idx]
                
            key_matches.append(x)
        
        return key_matches
    
    __getitem__ = query
    __setitem__ = add_entry

In [46]:
db = EncryptedStringDB(bob, alice, secure_worker)

In [47]:
# db.add_entry("bob", "tjdnf175")
# db.add_entry("alice", "tkfka175")
# db.add_entry("sec_work", "insecure")
db["bob"] = "tjdnf175"
db["alice"] = "tkfka175"
db["sec_work"] = "insecure"

In [48]:
# print(repr(db.query("bob")))
# print(repr(db.query("alice")))
# print(repr(db.query("sec_work")))
print(repr(db["bob"]))
print(repr(db["alice"]))
print(repr(db["sec_work"]))

'tjdnf175'
'tkfka175'
'insecure'


# Lesson: Encrypted Deep Learning in PyTorch

### Train a Model

In [49]:
from torch import nn
from torch import optim
import torch.nn.functional as F

# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

# A Toy Model
model = Net()

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)
        
train()

tensor(2.4000)
tensor(2.5587)
tensor(2.8824)
tensor(0.7130)
tensor(0.4339)
tensor(0.2742)
tensor(0.1787)
tensor(0.1123)
tensor(0.0680)
tensor(0.0404)
tensor(0.0240)
tensor(0.0146)
tensor(0.0093)
tensor(0.0062)
tensor(0.0045)
tensor(0.0034)
tensor(0.0027)
tensor(0.0022)
tensor(0.0018)
tensor(0.0015)


In [50]:
model(data)

tensor([[ 0.0163],
        [-0.0178],
        [ 0.9742],
        [ 1.0075]], grad_fn=<AddmmBackward>)

## Encrypt the Model and Data

In [51]:
encrypted_model = model.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [52]:
list(encrypted_model.parameters())

[Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:10904993546 -> alice:50924494026]
 	-> (Wrapper)>[PointerTensor | me:2605850933 -> bob:25367522906]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:40592808829 -> alice:35783348304]
 	-> (Wrapper)>[PointerTensor | me:20745875638 -> bob:300101761]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:82551338113 -> alice:84250780426]
 	-> (Wrapper)>[PointerTensor | me:77123306374 -> bob:98763949166]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:54188193453 -> alice:31601563354]
 	-> (Wrapper)>[PointerTensor | me:58509309716 -> bob:56855817350

In [53]:
encrypted_data = data.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [54]:
encrypted_data

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:46664584074 -> alice:33482466867]
	-> (Wrapper)>[PointerTensor | me:35655244115 -> bob:27711342930]
	*crypto provider: secure_worker*

In [55]:
encrypted_prediction = encrypted_model(encrypted_data)

In [56]:
encrypted_prediction.get().float_precision()

tensor([[ 0.0150],
        [-0.0190],
        [ 0.9730],
        [ 1.0050]])

# Lesson: Encrypted Deep Learning in Keras


## Step 1: Public Training

Welcome to this tutorial! In the following notebooks you will learn how to provide private predictions. By private predictions, we mean that the data is constantly encrypted throughout the entire process. At no point is the user sharing raw data, only encrypted (that is, secret shared) data. In order to provide these private predictions, Syft Keras uses a library called [TF Encrypted](https://github.com/tf-encrypted/tf-encrypted) under the hood. TF Encrypted combines cutting-edge cryptographic and machine learning techniques, but you don't have to worry about this and can focus on your machine learning application.

You can start serving private predictions with only three steps:
- **Step 1**: train your model with normal Keras.
- **Step 2**: secure and serve your machine learning model (server).
- **Step 3**: query the secured model to receive private predictions (client). 

Alright, let's go through these three steps so you can deploy impactful machine learning services without sacrificing user privacy or model security.

Huge shoutout to the Dropout Labs ([@dropoutlabs](https://twitter.com/dropoutlabs)) and TF Encrypted ([@tf_encrypted](https://twitter.com/tf_encrypted)) teams for their great work which makes this demo possible, especially: Jason Mancuso ([@jvmancuso](https://twitter.com/jvmancuso)), Yann Dupis ([@YannDupis](https://twitter.com/YannDupis)), and Morten Dahl ([@mortendahlcs](https://github.com/mortendahlcs)). 

_Demo Ref: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials_

## Train Your Model in Keras

To use privacy-preserving machine learning techniques for your projects you should not have to learn a new machine learning framework. If you have basic [Keras](https://keras.io/) knowledge, you can start using these techniques with Syft Keras. If you have never used Keras before, you can learn a bit more about it through the [Keras documentation](https://keras.io). 

Before serving private predictions, the first step is to train your model with normal Keras. As an example, we will train a model to classify handwritten digits. To train this model we will use the canonical [MNIST dataset](http://yann.lecun.com/exdb/mnist/).

We borrow [this example](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py) from the reference Keras repository.  To train your classification model, you just run the cell below.

In [57]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import Activation

batch_size = 128
num_classes = 10
epochs = 2

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(10, (3, 3), input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


Train on 60000 samples, validate on 10000 samples
Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Epoch 1/2


54912/60000 [==========================>...] - ETA: 1:46 - loss: 2.3027 - acc: 0.093 - ETA: 44s - loss: 2.3032 - acc: 0.067 - ETA: 32s - loss: 2.3019 - acc: 0.07 - ETA: 27s - loss: 2.3006 - acc: 0.11 - ETA: 23s - loss: 2.3004 - acc: 0.11 - ETA: 22s - loss: 2.2991 - acc: 0.13 - ETA: 20s - loss: 2.2981 - acc: 0.14 - ETA: 19s - loss: 2.2970 - acc: 0.14 - ETA: 18s - loss: 2.2954 - acc: 0.15 - ETA: 18s - loss: 2.2931 - acc: 0.17 - ETA: 17s - loss: 2.2909 - acc: 0.17 - ETA: 17s - loss: 2.2882 - acc: 0.18 - ETA: 16s - loss: 2.2846 - acc: 0.18 - ETA: 16s - loss: 2.2786 - acc: 0.18 - ETA: 16s - loss: 2.2723 - acc: 0.19 - ETA: 15s - loss: 2.2619 - acc: 0.20 - ETA: 15s - loss: 2.2480 - acc: 0.20 - ETA: 15s - loss: 2.2297 - acc: 0.22 - ETA: 15s - loss: 2.2107 - acc: 0.23 - ETA: 14s - loss: 2.1925 - acc: 0.23 - ETA: 14s - loss: 2.1708 - acc: 0.24 - ETA: 14s - loss: 2.1541 - acc: 0.25 - ETA: 14s - loss: 2.1360 - acc: 0.26 - ETA: 14s - loss: 2.1121 - acc: 0.27 - ETA: 14s - loss: 2.0944 - acc: 0.27 - 

60000/60000 [==============================] - ETA: 1s - loss: 0.7214 - acc: 0.767 - ETA: 1s - loss: 0.7197 - acc: 0.768 - ETA: 1s - loss: 0.7180 - acc: 0.768 - ETA: 1s - loss: 0.7164 - acc: 0.769 - ETA: 0s - loss: 0.7143 - acc: 0.769 - ETA: 0s - loss: 0.7121 - acc: 0.770 - ETA: 0s - loss: 0.7104 - acc: 0.771 - ETA: 0s - loss: 0.7083 - acc: 0.771 - ETA: 0s - loss: 0.7061 - acc: 0.772 - ETA: 0s - loss: 0.7039 - acc: 0.773 - ETA: 0s - loss: 0.7017 - acc: 0.774 - ETA: 0s - loss: 0.7001 - acc: 0.774 - ETA: 0s - loss: 0.6985 - acc: 0.775 - ETA: 0s - loss: 0.6966 - acc: 0.775 - ETA: 0s - loss: 0.6948 - acc: 0.776 - ETA: 0s - loss: 0.6928 - acc: 0.777 - ETA: 0s - loss: 0.6910 - acc: 0.777 - ETA: 0s - loss: 0.6892 - acc: 0.778 - ETA: 0s - loss: 0.6871 - acc: 0.778 - 16s 273us/sample - loss: 0.6859 - acc: 0.7793 - val_loss: 0.2704 - val_acc: 0.9177
Epoch 2/2


53248/60000 [=========================>....] - ETA: 16s - loss: 0.3013 - acc: 0.92 - ETA: 15s - loss: 0.3294 - acc: 0.90 - ETA: 15s - loss: 0.2988 - acc: 0.91 - ETA: 17s - loss: 0.2892 - acc: 0.91 - ETA: 17s - loss: 0.2794 - acc: 0.92 - ETA: 17s - loss: 0.2783 - acc: 0.91 - ETA: 17s - loss: 0.2920 - acc: 0.91 - ETA: 17s - loss: 0.2944 - acc: 0.91 - ETA: 17s - loss: 0.2873 - acc: 0.91 - ETA: 17s - loss: 0.2762 - acc: 0.92 - ETA: 17s - loss: 0.2738 - acc: 0.92 - ETA: 17s - loss: 0.2861 - acc: 0.91 - ETA: 17s - loss: 0.2850 - acc: 0.91 - ETA: 17s - loss: 0.2851 - acc: 0.91 - ETA: 17s - loss: 0.2805 - acc: 0.91 - ETA: 17s - loss: 0.2801 - acc: 0.91 - ETA: 17s - loss: 0.2806 - acc: 0.91 - ETA: 18s - loss: 0.2844 - acc: 0.91 - ETA: 18s - loss: 0.2819 - acc: 0.91 - ETA: 18s - loss: 0.2782 - acc: 0.91 - ETA: 18s - loss: 0.2775 - acc: 0.91 - ETA: 18s - loss: 0.2751 - acc: 0.91 - ETA: 18s - loss: 0.2746 - acc: 0.91 - ETA: 18s - loss: 0.2734 - acc: 0.91 - ETA: 18s - loss: 0.2728 - acc: 0.91 - ETA

60000/60000 [==============================] - ETA: 1s - loss: 0.2205 - acc: 0.931 - ETA: 1s - loss: 0.2204 - acc: 0.931 - ETA: 1s - loss: 0.2203 - acc: 0.932 - ETA: 1s - loss: 0.2199 - acc: 0.932 - ETA: 1s - loss: 0.2198 - acc: 0.932 - ETA: 1s - loss: 0.2193 - acc: 0.932 - ETA: 1s - loss: 0.2193 - acc: 0.932 - ETA: 1s - loss: 0.2190 - acc: 0.932 - ETA: 1s - loss: 0.2189 - acc: 0.932 - ETA: 1s - loss: 0.2188 - acc: 0.932 - ETA: 1s - loss: 0.2189 - acc: 0.932 - ETA: 1s - loss: 0.2187 - acc: 0.932 - ETA: 1s - loss: 0.2185 - acc: 0.932 - ETA: 0s - loss: 0.2186 - acc: 0.932 - ETA: 0s - loss: 0.2184 - acc: 0.932 - ETA: 0s - loss: 0.2184 - acc: 0.932 - ETA: 0s - loss: 0.2181 - acc: 0.932 - ETA: 0s - loss: 0.2178 - acc: 0.932 - ETA: 0s - loss: 0.2176 - acc: 0.932 - ETA: 0s - loss: 0.2178 - acc: 0.932 - ETA: 0s - loss: 0.2176 - acc: 0.933 - ETA: 0s - loss: 0.2175 - acc: 0.933 - ETA: 0s - loss: 0.2173 - acc: 0.933 - ETA: 0s - loss: 0.2169 - acc: 0.933 - ETA: 0s - loss: 0.2163 - acc: 0.933 - ETA

In [58]:
## Save your model's weights for future private prediction
model.save('short-conv-mnist.h5')

## Step 2: Load and Serve the Model

Now that you have a trained model with normal Keras, you are ready to serve some private predictions. We can do that using Syft Keras.

To secure and serve this model, we will need three TFEWorkers (servers). This is because TF Encrypted under the hood uses an encryption technique called [multi-party computation (MPC)](https://en.wikipedia.org/wiki/Secure_multi-party_computation). The idea is to split the model weights and input data into shares, then send a share of each value to the different servers. The key property is that if you look at the share on one server, it reveals nothing about the original value (input data or model weights).

We'll define a Syft Keras model like we did in the previous notebook. However, there is a trick: before instantiating this model, we'll run `hook = sy.KerasHook(tf.keras)`. This will add three important new methods to the Keras Sequential class:
 - `share`: will secure your model via secret sharing; by default, it will use the SecureNN protocol from TF Encrypted to secret share your model between each of the three TFEWorkers. Most importantly, this will add the capability of providing predictions on encrypted data.
 - `serve`: this function will launch a serving queue, so that the TFEWorkers can can accept prediction requests on the secured model from external clients.
 - `shutdown_workers`: once you are done providing private predictions, you can shut down your model by running this function. It will direct you to shutdown the server processes manually if you've opted to manually manage each worker.

If you want learn more about MPC, you can read this excellent [blog](https://mortendahl.github.io/2017/04/17/private-deep-learning-with-mpc/).

In [59]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation

import syft as sy
hook = sy.KerasHook(tf.keras)

## Model

As you can see, we define almost the exact same model as before, except we provide a `batch_input_shape`. This allows TF Encrypted to better optimize the secure computations via predefined tensor shapes. For this MNIST demo, we'll send input data with the shape of (1, 28, 28, 1). 
We also return the logit instead of softmax because this operation is complex to perform using MPC, and we don't need it to serve prediction requests.

In [60]:
num_classes = 10
input_shape = (1, 28, 28, 1)

model = Sequential()

model.add(Conv2D(10, (3, 3), batch_input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, name="logit"))

### Load Pre-trained Weights

With `load_weights` you can easily load the weights you have saved previously after training your model.

In [61]:
pre_trained_weights = 'short-conv-mnist.h5'
model.load_weights(pre_trained_weights)

## Step 3: Setup Your Worker Connectors

Let's now connect to the TFEWorkers (`alice`, `bob`, and `carol`) required by TF Encrypted to perform private predictions. For each TFEWorker, you just have to specify a host.

These workers run a [TensorFlow server](https://www.tensorflow.org/api_docs/python/tf/distribute/Server), which you can either manage manually (`AUTO = False`) or ask the workers to manage for you (`AUTO = True`). If choosing to manually manage them, you will be instructed to execute a terminal command on each worker's host device after calling `model.share()` below.  If all workers are hosted on a single device (e.g. `localhost`), you can choose to have Syft automatically manage the worker's TensorFlow server.

In [62]:
AUTO = True

alice = sy.TFEWorker(host='localhost:4010', auto_managed=AUTO)
bob = sy.TFEWorker(host='localhost:4011', auto_managed=AUTO)
carol = sy.TFEWorker(host='localhost:4012', auto_managed=AUTO)

## Step 4: Split the Model Into Shares

Thanks to `sy.KerasHook(tf.keras)` you can call the `share` method to transform your model into a TF Encrypted Keras model.

If you have asked to manually manage servers above then this step will not complete until they have all been launched. Note that your firewall may ask for Python to accept incoming connection.

In [63]:
model.share(alice, bob, carol)

INFO:tf_encrypted:Starting session on target 'grpc://localhost:4010' using config graph_options {
}



## Step 5: Launch 3 Servers

```
python -m tf_encrypted.player --config /tmp/tfe.config server0
python -m tf_encrypted.player --config /tmp/tfe.config server1
python -m tf_encrypted.player --config /tmp/tfe.config server2```

## Step 6: Serve the Model

Perfect! Now by calling `model.serve`, your model is ready to provide some private predictions. You can set `num_requests` to set a limit on the number of predictions requests served by the model; if not specified then the model will be served until interrupted.

In [64]:
model.serve(num_requests=3)

Served encrypted prediction 1 to client.
Served encrypted prediction 2 to client.
Served encrypted prediction 3 to client.


## Step 7: Run the Client

At this point open up and run the companion notebook: Section 4b - Encrytped Keras Client

## Step 8: Shutdown the Servers

Once your request limit above, the model will no longer be available for serving requests, but it's still secret shared between the three workers above. You can kill the workers by executing the cell below.

**Congratulations** on finishing Part 12: Secure Classification with Syft Keras and TFE!

In [65]:
model.shutdown_workers()

if not AUTO:
    process_ids = !ps aux | grep '[p]ython -m tf_encrypted.player --config /tmp/tfe.config' | awk '{print $2}'
    for process_id in process_ids:
        !kill {process_id}
        print("Process ID {id} has been killed.".format(id=process_id))

INFO:tf_encrypted:Please terminate the process on host 'localhost:4010'.
INFO:tf_encrypted:Please terminate the process on host 'localhost:4011'.
INFO:tf_encrypted:Please terminate the process on host 'localhost:4012'.


# Keystone Project - Mix and Match What You've Learned

Description: Take two of the concepts you've learned about in this course (Encrypted Computation, Federated Learning, Differential Privacy) and combine them for a use case of your own design. Extra credit if you can get your demo working with [WebSocketWorkers](https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials/advanced/websockets-example-MNIST) instead of VirtualWorkers! Then take your demo or example application, write a blogpost, and share that blogpost in #general-discussion on OpenMined's slack!!!

Inspiration:
- This Course's Code: https://github.com/Udacity/private-ai
- OpenMined's Tutorials: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials
- OpenMined's Blog: https://blog.openmined.org